In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy
import glob
import sys
from datetime import datetime

date="20200223-29"

In [ ]:
## Read files
#dataframes/{date}.csv
dataframe_file = f'dataframes/{date}_rtt.csv'
asn_location_file = f'Data/asn_location.csv'
dataframe_datatype = {0: str, 1: str, 2: str, 3: str, 4:str, 5:str, 6:str, 
                     7: str, 9: float, 10: float}
df = pd.read_csv(
                dataframe_file,
                sep=',',
                dtype= dataframe_datatype,
                    )

asn_datatype = { 0: int, 1:str, 2: str}
df_asn_location = pd.read_csv(
                        asn_location_file, 
                        sep=';',
                        index_col="id",
                        dtype = asn_datatype,
                    )

#df_asn_location.head(10)
#print(df.head())

In [ ]:
## Evaluate RTT by specified column

def eval_rtt_by_column(x):
    d = {}
    d['rtt_mean'] = x['rtt_mean'].mean()
    d['cnt'] = x['cnt'].sum()
    if x['cnt'].sum() != 0: 
        d['rtt_eval'] = (x['cnt'] * x['rtt_eval']).sum() / x['cnt'].sum()
    else:
        d['rtt_eval'] = None
    return pd.Series(d, index=['rtt_mean', 'rtt_eval', 'cnt', ])

def eval_rtt_by_column_with_count(x):
    d = {}
    d['rtt_mean'] = x['rtt_mean'].mean()
    d['cnt'] = x['cnt'].sum()
    if x['cnt'].sum() != 0: 
        d['rtt_eval'] = (x['cnt'] * x['rtt_mean']).sum() / x['cnt'].sum()
    else:
        d['rtt_eval'] = None
    d['group_count'] = x['cnt'].count()
    return pd.Series(d, index=['rtt_mean', 'rtt_eval', 'cnt', 'group_count'])

def dataframe_pretty(data_frame):
    data_frame = data_frame.rename(
        columns={"rtt_mean": "Průměrné RTT [ms]",
                "rtt_eval": "Vážené průměrné RTT [ms]",
                "cnt":"Počet dotazů",
                "q_percent": "Počet dotazů [%]"
                })
    return data_frame

def dataframe_pretty_without_percent(data_frame):
    data_frame = data_frame.rename(
        columns={"rtt_mean": "Průměrné RTT [ms]",
                "rtt_eval": "Vážené průměrné RTT [ms]",
                "cnt":"Počet dotazů",
                })
    return data_frame

def group_by_list(data_frame, group_list):
    data_frame = data_frame.groupby(group_list).apply(eval_rtt_by_column).reset_index()

    queries_all = data_frame['cnt'].sum()
    
    data_frame = data_frame[ group_list + ['rtt_mean',"rtt_eval",'cnt'] ].sort_values(
                                        by=['rtt_eval','cnt'], ascending=False)
    data_frame['q_percent'] = round(data_frame['cnt'] / (queries_all / 100 ),2)
    data_frame['rtt_mean'] = round(data_frame['rtt_mean'] * 1000)
    data_frame['rtt_eval'] = round(data_frame['rtt_eval'] * 1000)
    return data_frame.set_index(group_list)

# Rozdělení IP protolů

In [ ]:
df_ip = df.groupby(['ipv']).apply(eval_rtt_by_column).reset_index()

all_queries = df_ip['cnt'].sum()
df_ip['q_percent'] = df_ip['cnt'] / (all_queries / 100 )
df_ip_pretty = dataframe_pretty(df_ip)

df_ip_pretty

# Rozdělení anycastů
### IPv4

In [ ]:
df_anycast = df.groupby(['anycast', 'ipv']).apply(eval_rtt_by_column).reset_index()
df_anycast_4 = df_anycast.loc[df_anycast['ipv'] == "4"]
df_anycast_4 = df_anycast_4.set_index('anycast')
anycast_4_queries = df_anycast_4['cnt'].sum()
df_anycast_4['q_percent'] = df_anycast_4['cnt'] / (anycast_4_queries / 100 )
#sort_values(by=['cnt','rtt_mean','rtt_eval'], ascending=False)
df_anycast_4['rtt_mean'] = round(df_anycast_4['rtt_mean'] * 1000)
df_anycast_4['rtt_eval'] = round(df_anycast_4['rtt_eval'] * 1000)
df_anycast_4_pretty = dataframe_pretty(df_anycast_4)
#print(df_anycast_4_pretty.to_latex())
df_anycast_4_pretty


In [ ]:
df_anycast_region = group_by_list(df, ['anycast','ipv','region']).reset_index()
df_anycast_region = df_anycast_region.loc[(df_anycast_region['ipv'] == "4")
                                          & (df_anycast_region['anycast'] == "A")]
all_anycast_region = df_anycast_region['cnt'].sum()
df_anycast_region['q_percent'] = df_anycast_region['cnt'] / ( all_anycast_region / 100)
df_anycast_region_pretty = dataframe_pretty(df_anycast_region)
df_anycast_region_pretty = df_anycast_region_pretty.set_index(['anycast', 'ipv', 'region'])
#print(df_anycast_region_pretty.to_latex())
df_anycast_region_pretty

### IPv6

In [ ]:
df_anycast_6 = df_anycast.loc[df_anycast['ipv'] == "6"]
df_anycast_6 = df_anycast_6.set_index('anycast')
anycast_6_queries = df_anycast_6['cnt'].sum()
df_anycast_6['q_percent'] = df_anycast_6['cnt'] / (anycast_6_queries / 100 )
#sort_values(by=['cnt','rtt_mean','rtt_eval'], ascending=False)
df_anycast_6['rtt_mean'] = round(df_anycast_6['rtt_mean'] * 1000)
df_anycast_6['rtt_eval'] = round(df_anycast_6['rtt_eval'] * 1000)
df_anycast_6_pretty = dataframe_pretty(df_anycast_6)
#print(df_anycast_6_pretty.to_latex())
df_anycast_6_pretty

## Seřazené  regiony podle průměrného RTT a počtu dotazů

In [ ]:
queries = df.groupby(['region']).apply(eval_rtt_by_column).reset_index()

#print(queries_all.head())
queries_all = queries['cnt'].sum()
queries_region = queries[['region','rtt_mean',"rtt_eval",'cnt']].sort_values(by=['rtt_eval','cnt'], ascending=False)
queries_region['q_percent'] = queries_region['cnt'] / (queries_all / 100 )
queries_region['rtt_mean'] = round(queries_region['rtt_mean'] * 1000)
queries_region['rtt_eval'] = round(queries_region['rtt_eval'] * 1000)
queries_region_pretty = queries_region.rename(columns={"region": "Původce dotazu", "rtt_mean": "Průměrné RTT [ms]","rtt_eval": "Vážený průměrné RTT [ms]","cnt":"Počet dotazů","q_percent": "Počet dotazů [%]"})

queries_tmp = queries
queries_tmp['tmp'] = queries['rtt_eval'] * queries['cnt']
rtt_eval = round((queries_tmp['tmp'].sum() / queries_tmp['cnt'].sum()) * 1000)


print(f'Vážený průměr celé CZ domény je {rtt_eval} ms.')
print(f'Rozdělení dotazů mezi regiony s RTT (region zašťiťuje dotaz ze země původu):\n')
#print(queries_region.set_index("Původce dotazu"))
#print(queries_region_pretty.set_index("Původce dotazu").to_latex())

queries_region_pretty.set_index("Původce dotazu").style

## Seřazené  subregiony podle průměrného RTT a počtu dotazů

In [ ]:
queries_subregion = df.groupby(['sub_region']).apply(eval_rtt_by_column).reset_index()
#print(queries_all.head())
queries_all = queries_subregion['cnt'].sum()
queries_subregion = queries_subregion[['sub_region','rtt_mean', 'rtt_eval','cnt']].sort_values(by=['rtt_eval','cnt'], ascending=False)
queries_subregion['q_percent'] = queries_subregion['cnt'] / (queries_all / 100 )
queries_subregion['rtt_mean'] = round(queries_subregion['rtt_mean'] * 1000)
queries_subregion['rtt_eval'] = round(queries_subregion['rtt_eval'] * 1000)
queries_subregion_pretty = queries_subregion.rename(columns={"sub_region": "Původce dotazu", "rtt_mean": "Průměrné RTT [ms]","rtt_eval": "Vážený průměr RTT [ms]","cnt":"Počet dotazů","q_percent": "Počet dotazů [%]"})

print(f'Rozdělení dotazů mezi regiony s RTT (region zašťiťuje dotaz ze země původu):\n')
#print(queries_region.set_index("Původce dotazu"))
#print(queries_subregion_pretty.set_index("Původce dotazu").to_latex())

queries_subregion_pretty.set_index("Původce dotazu").style

## Státy v Asii

In [ ]:
queries_country = df.groupby(['region','sub_region','country_name']).apply(eval_rtt_by_column).reset_index()
df_country = queries_country[['region','sub_region','country_name','rtt_mean','rtt_eval','cnt']]
df_country = df_country.loc[df_country['region'] == "Asia"]
asia_queries = df_country['cnt'].sum()
df_country['q_percent'] = df_country['cnt'] / (asia_queries / 100 )
df_country['rtt_mean'] = round(df_country['rtt_mean'] * 1000)
df_country['rtt_eval'] = round(df_country['rtt_eval'] * 1000)
df_country_pretty = df_country.sort_values(by=['cnt','rtt_eval'], ascending=False).head(20)

traffic_percent = round(df_country_pretty['q_percent'].sum())
traffic_rtt = round(df_country_pretty['rtt_eval'].mean())
df_country_pretty = df_country_pretty.rename(columns={"country_name": "Původce dotazu", "rtt_mean": "Průměrné RTT [ms]", "rtt_eval": "Vážený průměr RTT [ms]", "cnt":"Počet dotazů","q_percent": "Počet dotazů [%]"})

print(f'Rozdělení dotazů v regionu Asia s RTT (region zašťiťuje dotaz ze země původu):\n')
#print(df_country_pretty.set_index("Původce dotazu"))

print(f'Těchto 10 statů generuje {traffic_percent} % provozu v Asii s váženým průměrným RTT {traffic_rtt} ms')
print(f'Mezi státy Asie je počítáno například Turecko nebo Izrael (Western Asia).')
df_country_pretty.set_index("Původce dotazu").reset_index().style

In [ ]:
queries_country = df.groupby(['region','sub_region','country_name']).apply(eval_rtt_by_column).reset_index()
df_country = queries_country[['region','sub_region','country_name','rtt_mean','rtt_eval','cnt']]
df_country = df_country.loc[df_country['region'] == "Asia"]
asia_queries = df_country['cnt'].sum()
df_country['q_percent'] = df_country['cnt'] / (asia_queries / 100 )
df_country['rtt_mean'] = round(df_country['rtt_mean'] * 1000)
df_country['rtt_eval'] = round(df_country['rtt_eval'] * 1000)

# Get names of indexes for which column Age has value 30
indexNames = df_country[ (df_country['sub_region'] == "Western Asia")
                        | (df_country['sub_region'] == "Central Asia") ].index
 
# Delete these row indexes from dataFrame
df_asia = df_country.drop(indexNames)

select_asia_queries = df_asia['cnt'].sum()
df_asia['q_percent'] = df_asia['cnt'] / (select_asia_queries / 100 )

df_asia_pretty = df_asia.sort_values(by=['cnt','rtt_mean','rtt_eval'], ascending=False).head(10)
traffic_asia_percent = round(df_asia_pretty['q_percent'].sum())
traffic_asia = df_asia_pretty['cnt'].sum()
traffic_asia_rtt = round(df_asia_pretty['rtt_eval'].mean())

## Eval avg RTT for asia
queries_asia_tmp = df_asia
queries_asia_tmp['tmp'] = df_asia['rtt_eval'] * df_asia['cnt']
asia_rtt_eval = round((queries_asia_tmp['tmp'].sum() / df_asia['cnt'].sum()))


df_asia_pretty = df_asia_pretty[["country_name", "sub_region", "rtt_mean","rtt_eval", "cnt", "q_percent"]]
#traffic_percent = round(df_country_pretty['q_percent'].sum())
#traffic_rtt = round(df_country_pretty['rtt_mean'].mean())
df_asia_pretty = df_asia_pretty.rename(columns={"country_name": "Původce dotazu", "rtt_mean": "Průměrné RTT [ms]",
                                                "rtt_eval" : "Vážený průměr RTT [ms]","cnt":"Počet dotazů",
                                                "q_percent": "Počet dotazů [%]"})

print(f'Rozdělení dotazů v regionu Asia (bez blízkého východu) s RTT (region zašťiťuje dotaz ze země původu):\n')

traffic_all_asia = round(traffic_asia / (queries_all / 100 ))

print(f'Těchto 10 statů generuje {traffic_asia_percent} % provozu v Asii s váženým průměrným RTT {asia_rtt_eval} ms.')
print(f'Z celého provozu CZ domény to je {traffic_all_asia} %')
print(f'Vážený průměr celé CZ domény je {rtt_eval} ms.')
#print(f'Mezi státy Asie je počítáno například Turecko nebo Izrael (Western Asia).')
#print(df_asia_pretty.to_latex())
df_asia_pretty.set_index("Původce dotazu").reset_index().style

In [ ]:
df_queries_server = df.groupby(["server","region","sub_region"]).apply(eval_rtt_by_column).reset_index()

df_queries_server = df_queries_server.loc[(df_queries_server['region'] == "Asia") & (df_queries_server['sub_region'] != "Western Asia") & (df_queries_server['sub_region'] != "Central Asia")]
df_queries_server = df_queries_server.groupby(["server"]).apply(eval_rtt_by_column).reset_index()
server_queries = df_queries_server['cnt'].sum()
df_queries_server['q_percent'] = df_queries_server['cnt'] / (server_queries / 100 )
#sort_values(by=['cnt','rtt_mean','rtt_eval'], ascending=False)
df_queries_server['rtt_mean'] = round(df_queries_server['rtt_mean'] * 1000)
df_queries_server['rtt_eval'] = round(df_queries_server['rtt_eval'] * 1000)
df_queries_server_pretty = df_queries_server.sort_values(by=['cnt','rtt_eval'], ascending=False)
traffic_percent = round(df_queries_server_pretty['q_percent'].sum())
traffic_rtt = round(df_queries_server_pretty['rtt_eval'].mean())
df_queries_server_pretty = df_queries_server_pretty.rename(columns={"rtt_mean": "Průměrné RTT [ms]", "rtt_eval": "Vážený průměr RTT [ms]", "cnt":"Počet dotazů","q_percent": "Počet dotazů [%]"})
print("Které DNS lokality odbavují provoz z Asie")
print("Z důvodu citlivosti údajů nejsou tyto záznamy vypsány.")
#df_queries_server_pretty.set_index("server").reset_index().style

In [ ]:
europa_server_list = ['<Hostnamy DNS serveru. Z duvodu citlivosti dat tyto zaznamy byly odstraneny>']
df_queries_server_tokyo_servers = df_queries_server.loc[(df_queries_server['server'] == "tokyo_01") | (df_queries_server['server'] == "tokyo_02")]
tokyo_servers = round(df_queries_server_tokyo_servers['q_percent'].sum())
df_queries_server_europa = df_queries_server.loc[(df_queries_server['server'].isin(europa_server_list))]
europa_p = round(df_queries_server_europa['q_percent'].sum())

print(f'Počet dotazů původem z Asie (bez blízkého východu), které obslouží servery v Tokyu je {tokyo_servers} %.')
print(f'Počet dotazů původem z Asie (bez blízkého východu), které obslouží Evropské servery je {europa_p} %.')


O Asii víme:
 * Špatné RTT oproti průměru 
 * Kolik provozu generuje
 * Kterých 10 států dělá většinu provozu
 * Které servery odbavují provoz a jejic procentuelní rozdělení
 
Nabízí se tedy otázka, kde vybudovat nový DNS server? 

In [ ]:
df_queries_asn = df.loc[(df['region'] == "Asia") & (df['sub_region'] != "Western Asia") & (df['sub_region'] != "Central Asia")]
df_queries_asn = df_queries_asn.groupby(["asn","region",
                                         "sub_region",
                                         "country_name"]).apply(eval_rtt_by_column).reset_index()

all_asn_queries = df_queries_asn['cnt'].sum()
df_queries_asn['q_percent'] = df_queries_asn['cnt'] / (all_asn_queries / 100 )
df_queries_asn['rtt_mean'] = round(df_queries_asn['rtt_mean'] * 1000)
df_queries_asn['rtt_eval'] = round(df_queries_asn['rtt_eval'] * 1000)
mean_traffic = df_queries_asn['cnt'].mean()
# Calculeta evaluate RTT of those who are above average number of queries at Asia
df_queries_asn_tmp = df_queries_asn
df_queries_asn_tmp['tmp'] = df_queries_asn['rtt_eval'] * df_queries_asn['cnt']
asn_rtt_eval = round((df_queries_asn_tmp['tmp'].sum() / df_queries_asn['cnt'].sum()))

df_queries_asn_cnt = df_queries_asn[['asn','country_name','rtt_mean', 'rtt_eval', 'cnt', 'q_percent']]
df_queries_asn_pretty = df_queries_asn_cnt.sort_values(by=['cnt', 'rtt_eval'], ascending=False)
df_queries_asn_pretty = df_queries_asn_pretty.rename(columns={"country_name": "Původce dotazu (Země)", 
                                                "rtt_mean": "Průměrné RTT [ms]", 
                                                "rtt_eval" : "Vážený průměr RTT [ms]", 
#                                                "rtt_count": "Počet vzorků pro výpočet RTT",
                                                "cnt":"Počet dotazů", "q_percent": "Počet dotazů [%]"})


top_number=10
asn_top_traffic= df_queries_asn_pretty.set_index('asn').head(top_number)['Počet dotazů [%]'].sum()
print(f'Tabulka {top_number} ASN s nejvíce počty dotazů z Asie')
print(f'Těchto {top_number} generuje {asn_top_traffic} %')
#print(df_queries_asn_pretty.set_index('asn').head(top_number).to_latex())
df_queries_asn_pretty.set_index('asn').head(top_number)



In [ ]:
df_queries_asn_location = df_queries_asn
# Prepare columns and indexes
df_queries_asn_location= df_queries_asn_location[['asn','country_name', 'cnt','rtt_mean', 'rtt_eval']]
df_queries_asn_location = df_queries_asn_location.set_index(['asn'])
df_asn_location_index = df_asn_location.set_index(['asn'])
# Join with df that contains all Asia asn
df_queries_asn_location = df_queries_asn_location.join(df_asn_location_index, on='asn', how='left').reset_index()
#df_queries_asn_location.loc[(df_queries_asn_location['asn'] == "4134")]
# Group by location and count number of queries that will hypoteticly capture and number of ASN that are location
df_datacenters = df_queries_asn_location.groupby(
                ['location']).apply(eval_rtt_by_column_with_count)[['cnt','group_count']]
df_datacenters_pretty = df_datacenters.sort_values(by=['cnt', 'group_count'], ascending=False).reset_index()
df_datacenters_pretty =  df_datacenters_pretty.rename(columns={"location": "Veřejný propojovací uzel", 
                                                "cnt":"Hypotetický počet dotazů", 
                                                "group_count": "Počet společných ASN",
                                                })

top_number = 15
decix_queries = df_queries_server.loc[df_queries_server['server'] == "decix-cz"].iloc[0,3]
hypo_decix = df_datacenters.reset_index()
hypo_decix = hypo_decix.loc[hypo_decix['location'] == "DE-CIX Frankfurt: DE-CIX Frankfurt Peering LAN"].iloc[0,1] - decix_queries

print(f'Top {top_number} lokalit s největším hypoteticky odbavenými dotazy z Asie.')
print(f'U uzlu DE-CIX Frankfurt jsou započítané i dotazy, které tam jsou již z Asie odbaveny!')
print(f'Decix Frankfurt ve dne {date} odbavil {decix_queries} dotazů.\n')
print(f'Decix Frankfurt bez již započtených odbavených dotazů by hypoteticky navíc odbavil:')
print(f'{hypo_decix}')

#print(df_datacenters_pretty.head(top_number).to_latex())
df_datacenters_pretty.head(top_number)

In [ ]:
df_queries_asn_location = df_queries_asn
# Prepare columns and indexes
df_queries_asn_location= df_queries_asn_location[['asn','country_name', 'cnt','rtt_mean', 'rtt_eval']]
df_queries_asn_location = df_queries_asn_location.set_index(['asn'])
df_top_asn = numpy.unique(df_queries_asn_pretty.set_index('asn').head(top_number).index.to_list())
df_asn_location_index = df_asn_location[df_asn_location['asn'].isin(df_top_asn)]
df_asn_location_index = df_asn_location_index.set_index(['asn'])
# Join with df that contains all Asia asn
df_queries_asn_location = df_queries_asn_location.join(df_asn_location_index, on='asn', how='left').reset_index()
#df_queries_asn_location.loc[(df_queries_asn_location['asn'] == "4134")]
# Group by location and count number of queries that will hypoteticly capture and number of ASN that are location
df_datacenters = df_queries_asn_location.groupby(
                ['location']).apply(eval_rtt_by_column_with_count)[['cnt','group_count']]
df_datacenters_pretty = df_datacenters.sort_values(by=['cnt', 'group_count'], ascending=False).reset_index()
df_datacenters_pretty =  df_datacenters_pretty.rename(columns={"location": "Veřejný propojovací uzel", 
                                                "cnt":"Hypotetický počet dotazů", 
                                                "group_count": "Počet společných ASN",
                                                })

top_number = 15
decix_queries = df_queries_server.loc[df_queries_server['server'] == "decix-cz"].iloc[0,3]
hypo_decix = df_datacenters.reset_index()
hypo_decix = hypo_decix.loc[hypo_decix['location'] == "DE-CIX Frankfurt: DE-CIX Frankfurt Peering LAN"].iloc[0,1] - decix_queries

print(f'Top {top_number} lokalit s největším hypoteticky odbavenými dotazy z Asie.')
print(f'U uzlu DE-CIX Frankfurt jsou započítané i dotazy, které tam jsou již z Asie odbaveny!')
print(f'Decix Frankfurt ve dne {date} odbavil {decix_queries} dotazů.\n')
print(f'Decix Frankfurt bez již započtených odbavených dotazů by hypoteticky navíc odbavil:')
print(f'{hypo_decix}')

#print(df_datacenters_pretty.head(top_number).to_latex())
df_datacenters_pretty.head(top_number)

In [ ]:
df_anycast_region = group_by_list(df, ['anycast','ipv','region']).reset_index()
df_anycast_region = df_anycast_region.loc[(df_anycast_region['ipv'] == "4") & (df_anycast_region['region'] == "Asia")]
all_anycast_region = df_anycast_region['cnt'].sum()
df_anycast_region['q_percent'] = df_anycast_region['cnt'] / ( all_anycast_region / 100)
df_anycast_region_pretty = dataframe_pretty(df_anycast_region)
#print(df_anycast_region_pretty.to_latex())
df_anycast_region_pretty

In [ ]:
df_cznic_locations = df_asn_location[(df_asn_location['asn'] == "25192") | (df_asn_location['asn'] == "200070")]
df_cznic_locations = df_cznic_locations.set_index('location')
df_cznic_locations = df_cznic_locations.drop(columns=['asn'])
#df_queries_asn_location.head()
df_cznic_above_asia = df_cznic_locations.join(df_queries_asn_location.set_index(['location']),
                                              on='location', how='left').reset_index()
df_cznic_above_asia_pretty = df_cznic_above_asia.groupby(
                ['location', 'asn']).apply(eval_rtt_by_column)
df_cznic_above_asia_pretty = dataframe_pretty_without_percent(df_cznic_above_asia_pretty)

print("Seznam lokalit kde může být navázán peering s ASN, které generují většinový provoz z Asie.")
#print(df_cznic_above_asia_pretty.to_latex())
df_cznic_above_asia_pretty